In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import copy

In [2]:
tf.config.experimental_run_functions_eagerly(True)
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
df = pd.read_csv("sudoku.csv")

In [4]:
df.head()

,quizzes,solutions
0,0043002090050090010700600430060020871900074000...,8643712593258497619712658434361925871986574322...
1,0401000501070039605200080000000000170009068008...,3461792581875239645296483719658324174729168358...
2,6001203840084590720000060050002640300700800069...,6951273841384596727248369158512647392739815469...
3,4972000001004000050000160986203000403009000000...,4972583161864397252537164986293815473759641828...
4,0059103080094030600275001000300002010008200070...,4659123781894735623275681497386452919548216372...


In [5]:
number_of_samples = 250000
unsolved = df['quizzes'].iloc[:number_of_samples].values
solved = df['solutions'].iloc[:number_of_samples].values

In [6]:
X = []
X2 = []
y = []

In [7]:
for sudoku in unsolved:
    np_unsolved = np.array([int(x) for x in sudoku]).reshape(9,9,1)
    X.append(np_unsolved)
    
for sudoku in solved:
    np_solved = np.array([int(y) for y in sudoku]).reshape(81,1) -1
    y.append(np_solved)
    np_solved_x = np.array([int(x) for x in sudoku]).reshape(9,9,1)
    X2.append(np_solved_x)

In [8]:
#normal approach
np_X = np.array(X)
np_X = np_X/9
np_X -= 0.5

#second approach
np_X2 = np.array(X2)
np_X2 = np_X2/9
np_X2 -= 0.5


In [9]:
np_y = np.array(y)

In [ ]:
#del(X)
#del(y)
#del(solved)
#del(unsolved)

In [10]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [12]:
X_train, X_test, y_train, y_test = train_test_split(np_X, np_y, test_size=0.2)

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Reshape, BatchNormalization, Activation, Input,Dropout
from tensorflow.keras.optimizers import Adam

In [ ]:
input_shape = (9,9,1)
batch_size = 128

In [ ]:
model = Sequential()

model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same', input_shape=input_shape))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(128, kernel_size=(1,1), activation='relu', padding='same'))

model.add(Flatten())
model.add(Dense(81*9))
model.add(Reshape((-1, 9)))
model.add(Activation('softmax'))

adam = Adam(learning_rate=0.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.backend import set_value

In [ ]:
early_stop = EarlyStopping(monitor='val_loss',patience=1, restore_best_weights=True)

In [ ]:
epochs =  [1, 2, 3, 4, 6, 8, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]  # epochs for each round
digits =  [1, 2, 4, 6, 9, 12, 16, 20, 25,30, 35, 40, 45, 50, 55]  # digit to pull off

In [ ]:
i=1
for _epochs, _delete in zip(epochs, digits):
    print(f'Pass n° {i} ...')
    i += 1
    model.fit(delete_digits(X_train,_delete), y_train, 
              validation_data=(delete_digits(X_test,_delete),y_test),
              epochs=_epochs, batch_size=16, callbacks=[early_stop])
    

In [13]:
def delete_digits(np_X, digits_to_delete=1):
    """
    This function is used to create sudoku quizzes from solutions
    
    Parameters
    ----------
    X (np.array), shape (?, 9, 9, 9|10): input solutions grids.
    n_delet (integer): max number of digit to suppress from original solutions
    
    Returns
    -------
    grids: np.array of grids to guess in one-hot way. Shape: (?, 9, 9, 10)
    """
    np_boards = np_X.copy()
    for np_board in np_boards:
        np_board.flat[np.random.randint(0, 81, digits_to_delete)] = - 0.5  # generate blanks (replace = True)
    return np_boards

In [14]:
from tensorflow.keras.models import load_model

In [15]:
#tf.keras.models.save_model(model, "CNN2.hp5", save_format="h5")

NameError: name 'model' is not defined

In [21]:
model = tf.keras.models.load_model("CNN.hp5", compile=False)

In [20]:
model2 =  tf.keras.models.load_model("CNN2.hp5", compile=False)

In [22]:
def normalize(np_a):
    
    return (np_a/9)-0.5

In [23]:
def denormalize(np_a):
    
    return (np_a+0.5)*9

In [48]:
def solve_batch(np_boards, np_solved_boards, model):
    np_results = np.empty(np_boards.shape[0],dtype=np.bool_)
    for i in np.ndindex(np_boards.shape[:1]):
        predictions = model.predict(np_boards[i].reshape(1,9,9,1))[0]
        solved = np.argmax(predictions, axis=1).reshape((9,9)) +1
        np_results[i] = compare_sudoku(solved, (np_solved_boards[i]+1).reshape(9,9))
        print(f"Solved sudoku number: {i}")
    correct_boards = np_results.sum()
    acc = correct_boards/np_boards.shape[0]
    return correct_boards

In [25]:
def solve_human_batch(np_boards, np_solved_boards, model):
    np_results = np.empty(np_boards.shape[0],dtype=np.bool_)
    for i in np.ndindex(np_boards.shape[:1]):
        solved = solve_human_approach(np_boards[i], model)
        np_results[i] = compare_sudoku(solved, (np_solved_boards[i]+1).reshape(9,9))
        print(f"Solved sudoku number: {i}")
    correct_boards = np_results.sum()
    acc = correct_boards/np_boards.shape[0]
    return correct_boards

In [26]:
def solve_human_approach (np_sample_board, model): 
    
    #np_board = np.copy(np_sample_board)
    np_board = np.copy(np_sample_board)
    #get number of blank in sudoku board
    #blanks_count = 81 - np.count_nonzero(np_board)
    
    #for _ in range(blanks_count):
    while(1):
        np_board_4d = np_board.reshape((1,9,9,1))
        predictions = model.predict(np_board_4d)[0]
        predictions_int = np.argmax(predictions, axis=1).reshape((9,9))+1           #change range of digits from <0,8> to <1,9>
        propabilities = np.around(np.max(predictions, axis=1).reshape((9,9)), 2) 
                        
        #get position with the highest propability
        np_board = denormalize(np_board).reshape((9,9))
        zeros = (np_board == 0)
        if (zeros.sum() == 0):
            break
        blanks_indices = np.where(zeros)
        blanks_indices = np.array([(i,j) for i,j in zip(blanks_indices[0], blanks_indices[1])])
        position_to_fill = blanks_indices[propabilities[zeros].argmax()]
        #fill position with the highest propability
        value_to_fill = predictions_int[position_to_fill[0], position_to_fill[1]]
        
        np_board[position_to_fill[0], position_to_fill[1]] = value_to_fill
        np_board = normalize(np_board)
    return np_board

In [27]:
def compare_sudoku (np_sudoku1, np_sudoku2):
    #True/False array
    comparision = np_sudoku1 == np_sudoku2
    #print(np.count_nonzero(comparision))
    return comparision.all()

In [49]:
x = solve_human_batch(X_test[:100], y_test[:100], model2)
x

Solved sudoku number: (0,)
Solved sudoku number: (1,)
Solved sudoku number: (2,)
Solved sudoku number: (3,)
Solved sudoku number: (4,)
Solved sudoku number: (5,)
Solved sudoku number: (6,)
Solved sudoku number: (7,)
Solved sudoku number: (8,)
Solved sudoku number: (9,)
Solved sudoku number: (10,)
Solved sudoku number: (11,)
Solved sudoku number: (12,)
Solved sudoku number: (13,)
Solved sudoku number: (14,)
Solved sudoku number: (15,)
Solved sudoku number: (16,)


KeyboardInterrupt: 

In [50]:
x = solve_batch(X_test[:100], y_test[:100], model)
x

Solved sudoku number: (0,)
Solved sudoku number: (1,)
Solved sudoku number: (2,)
Solved sudoku number: (3,)
Solved sudoku number: (4,)
Solved sudoku number: (5,)
Solved sudoku number: (6,)
Solved sudoku number: (7,)
Solved sudoku number: (8,)
Solved sudoku number: (9,)
Solved sudoku number: (10,)
Solved sudoku number: (11,)
Solved sudoku number: (12,)
Solved sudoku number: (13,)
Solved sudoku number: (14,)
Solved sudoku number: (15,)
Solved sudoku number: (16,)
Solved sudoku number: (17,)
Solved sudoku number: (18,)
Solved sudoku number: (19,)
Solved sudoku number: (20,)
Solved sudoku number: (21,)
Solved sudoku number: (22,)
Solved sudoku number: (23,)
Solved sudoku number: (24,)
Solved sudoku number: (25,)
Solved sudoku number: (26,)
Solved sudoku number: (27,)
Solved sudoku number: (28,)
Solved sudoku number: (29,)
Solved sudoku number: (30,)
Solved sudoku number: (31,)
Solved sudoku number: (32,)
Solved sudoku number: (33,)
Solved sudoku number: (34,)
Solved sudoku number: (35,)
So

0

In [45]:
y_test[0].reshape(9,9) + 1

array([[7, 2, 6, 5, 8, 3, 9, 4, 1],
       [4, 5, 1, 7, 6, 9, 3, 2, 8],
       [8, 9, 3, 1, 4, 2, 6, 7, 5],
       [5, 1, 2, 6, 3, 8, 4, 9, 7],
       [6, 7, 4, 9, 2, 5, 1, 8, 3],
       [9, 3, 8, 4, 7, 1, 5, 6, 2],
       [1, 4, 5, 8, 9, 7, 2, 3, 6],
       [3, 6, 7, 2, 5, 4, 8, 1, 9],
       [2, 8, 9, 3, 1, 6, 7, 5, 4]])

In [ ]:
compare_sudoku(x,(y_test[150]+1).reshape(9,9))